# Persuasion Techniques in Text of Memes

## Enironment Setup

##### Disk Setup

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# For emptying trash after each run
#from google.colab import auth
#auth.authenticate_user()
#from googleapiclient.discovery import build
#drive_service = build('drive', 'v3')
#drive_service.files().emptyTrash().execute()

In [3]:
!pwd
!nvidia-smi

/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/subtask1/transformer/training
Sun Jan 28 15:21:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  | 00000000:2B:00.0 Off |                  N/A |
|  0%   41C    P8              26W / 350W |      3MiB / 12288MiB |      0%      Default |
|                                         |                     

In [4]:
folder_name = "/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/" #"/content/drive/MyDrive/persuasion_technique_detection/"

##### Imports

In [5]:
#!pip install transformers datasets wandb evaluate accelerate -qU sklearn_hierarchical_classification sentencepiece

In [6]:
import gc
import re
import json
import numpy as np
import pandas as pd
import random
import torch
import subprocess
import json
import warnings
import shutil

In [7]:
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
from datasets import Dataset,load_dataset,DatasetDict,concatenate_datasets
import datasets
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
import os
from datasets import concatenate_datasets
from transformers import Trainer
from transformers import AutoTokenizer, DataCollatorWithPadding

In [8]:
AVAIL_GPUS = 0
if torch.cuda.is_available():
    device = torch.device("cuda")
    AVAIL_GPUS = torch.cuda.device_count()
    print(f'There are {AVAIL_GPUS} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3080 Ti


In [9]:
!nvidia-smi

Sun Jan 28 15:21:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  | 00000000:2B:00.0 Off |                  N/A |
|  0%   42C    P8              26W / 350W |      5MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Login to WandB

In [10]:
import wandb
import os

#wandb.login(relogin=True)
wandb.login()

# setup wandb environment variables
os.environ['WANDB_PROJECT'] = "subtask1_transformer_encoder_classification"
os.environ['WANDB_ENTITY'] = "tumnlp"
os.environ["WANDB_LOG_MODEL"]= "end"

wandb: Currently logged in as: mahmudfami (tumnlp). Use `wandb login --relogin` to force relogin


## Pre-trained Transformer Name

In [11]:
#checkpoint = "bert-base-cased"
#checkpoint="vinai/bertweet-base"
#checkpoint = "xlm-roberta-base"
#checkpoint = "xlnet-base-cased"
#checkpoint = "microsoft/deberta-v3-base"
#checkpoint = "albert-base-v2"
#checkpoint = "distilroberta-base"
#checkpoint = "microsoft/deberta-v3-base"
#checkpoint = "microsoft/deberta-v3-large"
#checkpoint= "bert-large-cased"
#checkpoint="huggyllama/llama-7b"
#checkpoint="xlm-roberta-base"
#checkpoint = "roberta-large"
checkpoint="vinai/bertweet-large"
#checkpoint ="jayanta/twitter-roberta-base-sentiment-sentiment-memes-30epcohs"
#checkpoint = "princeton-nlp/Sheared-LLaMA-1.3B"
#checkpoint = "albert-large-v2"

In [12]:
summary_dir_path = folder_name + "subtask1/transformer/summaries/hierarchical/summary_train_" + checkpoint.replace("/","_") + "_memes_transfer_gradient/"

## Data Preprocessing

In [13]:
# Initialize a counter for empty label samples
empty_label_counter = [0]

def filter_every_second_empty_label(sample):
    if len(sample['labels']) == 0:
        empty_label_counter[0] += 1

        return empty_label_counter[0] % 2 != 0
    return True

In [14]:
val_path=folder_name+"data/subtask1/validation.json"
train_path = folder_name+"data/subtask1/train.json"
test_path=folder_name+"data/subtask1/dev_unlabeled.json"
train_added_ptc = folder_name+"data/ptc/ptc_added_train.json"

#dataset_memes_2024_files = {"train": train_added_ptc,"validation": val_path} #PTC + MEMES!!!!
dataset_memes_2024_files = {"train": train_path,"validation": val_path} # ONLY MEMES!!!
val_files={"val":val_path}
test_files={"test":test_path}

dataset_memes_2024 = load_dataset("json",data_files=dataset_memes_2024_files)
#dataset_memes_2024 = dataset_memes_2024.filter(lambda x : len(x['labels']) != 0)
#dataset_memes_2024 = dataset_memes_2024.filter(filter_every_second_empty_label)

#dataset_val=load_dataset("json",data_files=val_files)
dataset_test=load_dataset("json",data_files=test_files)

In [15]:
lvl1_parents={"Appeal to authority":["Ethos","Logos"],
              "Appeal to fear/prejudice":["Pathos","Logos"],
              "Bandwagon":["Ethos","Logos"],
              "Black-and-white Fallacy/Dictatorship":["Logos"],
              "Causal Oversimplification":["Logos"],
              "Doubt":["Ethos"],
              "Exaggeration/Minimisation":["Pathos"],
              "Flag-waving":["Pathos","Logos"],
              "Glittering generalities (Virtue)":["Ethos"],
              "Loaded Language":["Pathos"],
              "Misrepresentation of Someone's Position (Straw Man)":["Logos"],
              "Name calling/Labeling":["Ethos"],
              "Obfuscation, Intentional vagueness, Confusion":["Logos"],
              "Presenting Irrelevant Data (Red Herring)":["Logos"],
              "Reductio ad hitlerum":["Ethos"],
              "Repetition":["Logos"],
              "Slogans":["Logos"],
              "Smears":["Ethos"],
              "Thought-terminating cliché":["Logos"],
              "Whataboutism":["Ethos","Logos"]}

lvl2_parents={"Appeal to authority":["Justification"],
              "Appeal to fear/prejudice":["Justification"],
              "Bandwagon":["Justification"],
              "Black-and-white Fallacy/Dictatorship":["Reasoning"],
              "Causal Oversimplification":["Reasoning"],
              "Doubt":["Ad Hominem"],
              "Exaggeration/Minimisation":[],
              "Flag-waving":["Justification"],
              "Glittering generalities (Virtue)":[],
              "Loaded Language":[],
              "Misrepresentation of Someone's Position (Straw Man)":["Reasoning"],
              "Name calling/Labeling":["Ad Hominem"],
              "Obfuscation, Intentional vagueness, Confusion":[],
              "Presenting Irrelevant Data (Red Herring)":["Reasoning"],
              "Reductio ad hitlerum":["Ad Hominem"],
              "Repetition":[],
              "Slogans":["Justification"],
              "Smears":["Ad Hominem"],
              "Thought-terminating cliché":["Reasoning"],
              "Whataboutism":["Ad Hominem","Reasoning"]}

lvl3_parents={"Appeal to authority":[],
              "Appeal to fear/prejudice":[],
              "Bandwagon":[],
              "Black-and-white Fallacy/Dictatorship":["Simplification"],
              "Causal Oversimplification":["Simplification"],
              "Doubt":[],
              "Exaggeration/Minimisation":[],
              "Flag-waving":["Justification"],
              "Glittering generalities (Virtue)":[],
              "Loaded Language":[],
              "Misrepresentation of Someone's Position (Straw Man)":["Distraction"],
              "Name calling/Labeling":[],
              "Obfuscation, Intentional vagueness, Confusion":[],
              "Presenting Irrelevant Data (Red Herring)":["Distraction"],
              "Reductio ad hitlerum":[],
              "Repetition":[],
              "Slogans":[],
              "Smears":[],
              "Thought-terminating cliché":["Simplification"],
              "Whataboutism":["Distraction"]}


def create_parent_labels(data):
  labels=data["labels"]
  persuasion=[]
  ethos=[]
  pathos=[]
  logos=[]
  ad_hominem=[]
  justification=[]
  reasoning=[]
  distraction=[]
  simplification=[]
  persuasion_or_not=[]

  if len(labels)>0:
    persuasion_or_not.append("Persuasion")
    for el in labels:

      if "Ethos" in lvl1_parents[el]:
        persuasion.append("Ethos")
        if len(lvl2_parents[el])>0 and "Ad Hominem" in lvl2_parents[el]:
          ethos.append("Ad Hominem")
          ad_hominem.append(el)
        else:
          ethos.append(el)
      if "Pathos" in lvl1_parents[el]:
        persuasion.append("Pathos")
        pathos.append(el)
      if "Logos" in lvl1_parents[el]:
        persuasion.append("Logos")
        if len(lvl2_parents[el])>0 and "Justification" in lvl2_parents[el]:
          logos.append("Justification")
          justification.append(el)
        elif len(lvl2_parents[el])>0 and "Reasoning" in lvl2_parents[el]:
          logos.append("Reasoning")
          reasoning.append(lvl3_parents[el][0])
          if lvl3_parents[el][0]=="Simplification":
            simplification.append(el)
          else:

            distraction.append(el)
        else:
          logos.append(el)
  else:
    persuasion_or_not.append("Not Persuasion")


  data["persuasion"]=list(set(persuasion))
  data["ethos"]=list(set(ethos))
  data["pathos"]=list(set(pathos))
  data["logos"]=list(set(logos))
  data["ad_hominem"]=list(set(ad_hominem))
  data["justification"]=list(set(justification))
  data["reasoning"]=list(set(reasoning))
  data["distraction"]=list(set(distraction))
  data["simplification"]=list(set(simplification))
  data["persuasion_or_not"]=persuasion_or_not
  return data

dataset_memes_2024=dataset_memes_2024.map(create_parent_labels)

In [16]:
dataset_memes_2024["train"][5]

{'labels': [],
 'text': "PUTIN'S SECRET CAMOUFLAGE ARMY",
 'link': 'https://www.facebook.com/photo/?fbid=1261268274677920&set=g.655806642139792',
 'id': '66402',
 'persuasion': [],
 'ethos': [],
 'pathos': [],
 'logos': [],
 'ad_hominem': [],
 'justification': [],
 'reasoning': [],
 'distraction': [],
 'simplification': [],
 'persuasion_or_not': ['Not Persuasion']}

In [17]:
dataset_memes_2024

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'link', 'id', 'persuasion', 'ethos', 'pathos', 'logos', 'ad_hominem', 'justification', 'reasoning', 'distraction', 'simplification', 'persuasion_or_not'],
        num_rows: 7000
    })
    validation: Dataset({
        features: ['labels', 'text', 'link', 'id', 'persuasion', 'ethos', 'pathos', 'logos', 'ad_hominem', 'justification', 'reasoning', 'distraction', 'simplification', 'persuasion_or_not'],
        num_rows: 500
    })
})

In [18]:
unique_labels=list(set([x for el in dataset_memes_2024["train"]["labels"] for x in el ]))
#unique_labels

In [19]:
persuasion_unique=[["Ethos","Pathos","Logos"]]
ethos_unique=[["Ad Hominem","Bandwagon","Appeal to authority","Glittering generalities (Virtue)"]]
pathos_unique=[["Exaggeration/Minimisation","Loaded Language","Flag-waving","Appeal to fear/prejudice"]]
logos_unique=[["Justification","Reasoning","Repetition","Obfuscation, Intentional vagueness, Confusion"]]
ad_hominem_unique=[["Name calling/Labeling","Doubt","Smears","Reductio ad hitlerum","Whataboutism"]]
justification_unique=[["Bandwagon","Appeal to authority","Flag-waving","Appeal to fear/prejudice","Slogans"]]
reasoning_unique=[["Distraction","Simplification"]]
distraction_unique=[["Misrepresentation of Someone's Position (Straw Man)","Presenting Irrelevant Data (Red Herring)","Whataboutism"]]
simplification_unique=[["Causal Oversimplification","Black-and-white Fallacy/Dictatorship","Thought-terminating cliché"]]
persuasion_or_not_unique=[["Not Persuasion","Persuasion"]]

### Preprocess Multi-Labels

In [20]:
mlb_persuasion = MultiLabelBinarizer()
mlb_persuasion.fit(persuasion_unique)

mlb_ethos = MultiLabelBinarizer()
mlb_ethos.fit(ethos_unique)

mlb_pathos = MultiLabelBinarizer()
mlb_pathos.fit(pathos_unique)

mlb_logos = MultiLabelBinarizer()
mlb_logos.fit(logos_unique)

mlb_ad_hominem = MultiLabelBinarizer()
mlb_ad_hominem.fit(ad_hominem_unique)

mlb_justification = MultiLabelBinarizer()
mlb_justification.fit(justification_unique)

mlb_reasoning=MultiLabelBinarizer()
mlb_reasoning.fit(reasoning_unique)

mlb_distraction=MultiLabelBinarizer()
mlb_distraction.fit(distraction_unique)

mlb_simplification=MultiLabelBinarizer()
mlb_simplification.fit(simplification_unique)

lb_persuasion_or_not=LabelEncoder()
lb_persuasion_or_not.fit(persuasion_or_not_unique[0])


LabelEncoder()

In [21]:
val_memes=dataset_memes_2024["validation"]

In [22]:
dataset_memes_2024=dataset_memes_2024.remove_columns(["labels"])

memes_persuasion_or_not=dataset_memes_2024.rename_column("persuasion_or_not","labels")
memes_persuasion=dataset_memes_2024.rename_column("persuasion","labels")
memes_ethos=dataset_memes_2024.rename_column("ethos","labels")
memes_pathos=dataset_memes_2024.rename_column("pathos","labels")
memes_logos=dataset_memes_2024.rename_column("logos","labels")
memes_ad_hominem=dataset_memes_2024.rename_column("ad_hominem","labels")
memes_justification=dataset_memes_2024.rename_column("justification","labels")
memes_reasoning=dataset_memes_2024.rename_column("reasoning","labels")
memes_distraction=dataset_memes_2024.rename_column("distraction","labels")
memes_simplification=dataset_memes_2024.rename_column("simplification","labels")

memes_persuasion = memes_persuasion.filter(lambda x : len(x['labels']) != 0)
memes_ethos = memes_ethos.filter(lambda x : len(x['labels']) != 0)
memes_pathos = memes_pathos.filter(lambda x : len(x['labels']) != 0)
memes_logos = memes_logos.filter(lambda x : len(x['labels']) != 0)
memes_ad_hominem = memes_ad_hominem.filter(lambda x : len(x['labels']) != 0)
memes_justification = memes_justification.filter(lambda x : len(x['labels']) != 0)
memes_reasoning = memes_reasoning.filter(lambda x : len(x['labels']) != 0)
memes_distraction = memes_distraction.filter(lambda x : len(x['labels']) != 0)
memes_simplification = memes_simplification.filter(lambda x : len(x['labels']) != 0)

In [23]:
for label in ["Not Persuasion","Persuasion"]:
    count = len(memes_persuasion_or_not['validation'].filter(lambda example: example["labels"] == [label]))
    print(f"Label: {label}, Count: {count}")

Label: Not Persuasion, Count: 88
Label: Persuasion, Count: 412


In [24]:
memes_persuasion["validation"]

Dataset({
    features: ['text', 'link', 'id', 'labels', 'ethos', 'pathos', 'logos', 'ad_hominem', 'justification', 'reasoning', 'distraction', 'simplification', 'persuasion_or_not'],
    num_rows: 412
})

In [25]:
memes_persuasion["validation"][5]

{'text': '\\ I left my parents. I left my brothers and sisters, my relatives, to gain knowledge and experience that will\\nhelp me free our territory from occupiers and invaders.\\\\n\\nZakhar, a Ukrainian Army recruit and former engineer.',
 'link': 'https://www.facebook.com/photo/?fbid=1276830019788412&set=g.655806642139792',
 'id': '66205',
 'labels': ['Pathos', 'Logos', 'Ethos'],
 'ethos': ['Ad Hominem'],
 'pathos': ['Flag-waving', 'Loaded Language'],
 'logos': ['Justification'],
 'ad_hominem': ['Name calling/Labeling'],
 'justification': ['Flag-waving'],
 'reasoning': [],
 'distraction': [],
 'simplification': [],
 'persuasion_or_not': ['Persuasion']}

#### Tokenize

In [26]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples,binarizer):
    encoding = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512) #change max length after done
    
    if isinstance(binarizer, LabelEncoder):
      encoding['labels'] = binarizer.transform(np.array(examples['labels']).ravel())
    else:
      encoding['labels'] = binarizer.transform(examples['labels']).astype(np.float32).tolist()

    return encoding

tokenized_persuasion = memes_persuasion.map(tokenize_function,fn_kwargs={"binarizer":mlb_persuasion} ,batched=True)

tokenized_ethos = memes_ethos.map(tokenize_function,fn_kwargs={"binarizer":mlb_ethos },batched=True)

tokenized_pathos = memes_pathos.map(tokenize_function,fn_kwargs={"binarizer":mlb_pathos} ,batched=True)

tokenized_logos = memes_logos.map(tokenize_function,fn_kwargs={"binarizer":mlb_logos},batched=True)

tokenized_ad_hominem = memes_ad_hominem.map(tokenize_function,fn_kwargs={"binarizer":mlb_ad_hominem },batched=True)

tokenized_justification = memes_justification.map(tokenize_function,fn_kwargs={"binarizer":mlb_justification} ,batched=True)

tokenized_reasoning = memes_reasoning.map(tokenize_function,fn_kwargs={"binarizer":mlb_reasoning},batched=True)

tokenized_distraction = memes_distraction.map(tokenize_function,fn_kwargs={"binarizer":mlb_distraction} ,batched=True)

tokenized_simplification = memes_simplification.map(tokenize_function,fn_kwargs={"binarizer":mlb_simplification} ,batched=True)

tokenized_persuasion_or_not = memes_persuasion_or_not.map(tokenize_function,fn_kwargs={"binarizer":lb_persuasion_or_not},batched=True)

Map:   0%|          | 0/2078 [00:00<?, ? examples/s]

## Training

In [27]:
def save_json(data, file_path):
    with open(file_path, 'w+') as f:
        json.dump(data, f)

def create_json_output(y_pred, y_true,binarizer):

    # Transfrom numeric labels into textual labels
    y_pred = binarizer.inverse_transform(y_pred)
    y_true = binarizer.inverse_transform(y_true)

    # Convert y_true and y_pred to the required JSON format
    if isinstance(binarizer,LabelEncoder):
      pred_json = [{"id": str(idx), "labels": [pred_row]} for idx, pred_row in enumerate(y_pred)]
      true_json = [{"id": str(idx), "labels": [true_row]} for idx, true_row in enumerate(y_true)]
    else:
      pred_json = [{"id": str(idx), "labels": pred_row} for idx, pred_row in enumerate(y_pred)]
      true_json = [{"id": str(idx), "labels": true_row} for idx, true_row in enumerate(y_true)]

    # Save to json files
    predictions_file = folder_name + f"subtask1/output_{node_name}/tmp/predictions.json"
    gold_labels_file= folder_name + f"subtask1/output_{node_name}/tmp/gold_labels.json"
    save_json(pred_json, predictions_file)
    save_json(true_json, gold_labels_file)
    return predictions_file, gold_labels_file

In [28]:
def calculate_hierarchical_metrics(predictions_file, gold_labels_file):
    # Run the scoring script
    scorer = folder_name + "subtask1/subtask_1_2a.py"
    command = f'python3 {scorer} --gold_file_path {gold_labels_file} --pred_file_path {predictions_file}'
    try:
      result = subprocess.run(command, shell=True, check=True , stdout=subprocess.PIPE, text=True)

    except subprocess.CalledProcessError as e:
      output = e.output
      print("error:",e)
    res=result.stdout
    output = res.strip()

    # Parse the output
    parts = output.split('\t')
    f1_h = float(parts[0].split('=')[1])
    prec_h = float(parts[1].split('=')[1])
    rec_h = float(parts[2].split('=')[1])

    return f1_h, prec_h, rec_h

In [29]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, binarizer,threshold=0.5):
    y_true = labels
    if isinstance(binarizer,LabelEncoder):
      y_pred=np.argmax(predictions, axis=1)
      f1 = f1_score(y_true, y_pred, average='binary')
    else:
      # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
      sigmoid = torch.nn.Sigmoid()
      probs = sigmoid(torch.Tensor(predictions))
      # next, use threshold to turn them into integer predictions
      y_pred = np.zeros(predictions.shape)
      y_pred[np.where(probs >= threshold)] = 1


      # compute metrics
      f1= f1_score(y_true=y_true, y_pred=y_pred, average='micro')

    # Create Json Output file
    predictions_file, gold_labels_file = create_json_output(y_pred, y_true,binarizer)
    accuracy = accuracy_score(y_true, y_pred)
    f1_h, prec_h, rec_h = calculate_hierarchical_metrics(predictions_file, gold_labels_file)

    # return as dictionary
    metrics = {'f1_hierarchical': f1_h,
               'precision_hierarchical': prec_h,
               'recall_hierarchical': rec_h,
               'f1': f1,
               'accuracy': accuracy,
               }
    print(metrics)
    return metrics

def prepare_compute_metrics(mlb,threshold):
  def compute_metrics(p: EvalPrediction):
    nonlocal mlb
    nonlocal threshold
    preds = p.predictions[0] if isinstance(p.predictions,tuple) else p.predictions
    labels=p.label_ids
    result = multi_label_metrics(
        predictions=preds,
        labels=labels,binarizer=mlb,threshold=threshold)
    return result
  return compute_metrics

In [30]:
def train():
    try:
        # Initialize a new wandb run
        wandb.init()

        # sweep agent inputs config with hyperparameters
        config = wandb.config

        # Use config to access the hyperparameters
        threshold = config.threshold
        learning_rate = config.learning_rate
        epochs = 3
        batch_size=4
        run_name = config.run_name+f"_{str(threshold)}threshold_{str(learning_rate)}learningRate"
        wandb.run.name = run_name

        training_args = TrainingArguments(
            report_to = 'wandb',                    # enable logging to W&B
            run_name = run_name+f"_{str(threshold)}threshold_{str(learning_rate)}_learningRate",          # name of the W&B run
            load_best_model_at_end = True,
            output_dir = output_folder,
            overwrite_output_dir = True,
            save_strategy = 'steps',
            evaluation_strategy = 'steps',
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            learning_rate = learning_rate,
            num_train_epochs = epochs,
            save_steps = 1 / (epochs*3),
            eval_steps = 1 / (epochs*3),
            logging_steps = 100,
            save_total_limit=1,
            metric_for_best_model = 'f1_hierarchical',
            gradient_accumulation_steps = 2
        )

        trainer = Trainer(
            model,
            training_args,
            train_dataset=tokenized_dataset["train"],
            eval_dataset=tokenized_dataset["validation"],
            data_collator=data_collator,
            tokenizer=tokenizer,
            compute_metrics=prepare_compute_metrics(mlb,threshold)
        )

        trainer.train()

        wandb.finish()
    except Exception as e:
        print(f"Error in training node {node_name}: {str(e)}")
    # empty trash of drive
    #drive_service.files().emptyTrash().execute()

In [31]:
# Search for float value before 'threshold'
def extract_threshold(string):
    match = re.search(r'(\d+\.\d+)threshold', string)
    if match:
        return float(match.group(1))
    else:
        return None

In [32]:
# Prepare nodes for loop
node_names = ["persuasion_or_not","persuasion","ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [persuasion_or_not_unique,persuasion_unique, ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
tokenized_node_datasets = [tokenized_persuasion_or_not,tokenized_persuasion,tokenized_ethos,tokenized_pathos,tokenized_logos,tokenized_ad_hominem,tokenized_justification,tokenized_reasoning,tokenized_simplification,tokenized_distraction]
mlb_nodes = [lb_persuasion_or_not,mlb_persuasion, mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]
# Prepare for transfer learning
transfer_learning = False
model_nodes = {
    "persuasion_or_not":None,
    "persuasion" : None,
    "ethos" : None,
    "pathos": None,
    "logos": None,
    "ad_hominem": None,
    "justification": None,
    "reasoning": None,
    "simplification": None,
    "distraction": None
}
parent_nodes = {
    "persuasion_or_not":"persuasion_or_not",
    "persuasion" : "persuasion_or_not",
    "ethos" : "persuasion",
    "pathos": "persuasion",
    "logos": "persuasion",
    "ad_hominem": "ethos",
    "justification": "logos",
    "reasoning": "logos",
    "simplification": "reasoning",
    "distraction": "reasoning"
}
threshold_nodes= {
    "persuasion_or_not": 0.5,
    "persuasion": 0.3,
    "ethos": 0.4,
    "pathos": 0.3,
    "logos": 0.5,
    "ad_hominem": 0.3,
    "justification": 0.4,
    "reasoning": 0.3,
    "simplification": 0.4,
    "distraction": 0.5
}

In [33]:
# Loop to train all nodes after each other

for i in range(len(node_names)):

  node_name = node_names[i]
  #threshold = threshold_nodes[node_name]
  unique = unique_node_labels[i]
  tokenized_dataset = tokenized_node_datasets[i]
  mlb = mlb_nodes[i]

  run_name=f'{checkpoint}-{node_name}-memes'.replace("/","_")
  sweep_name=f'sweep_{run_name}'


  output_folder = folder_name + f"subtask1/output_{node_name}"
  if not os.path.exists(output_folder+"/tmp"):
    os.makedirs(output_folder+"/tmp")
  

  # load pre-trained model weights of parent node
  model_name = model_nodes[parent_nodes[node_name]]

  if model_name != None and transfer_learning:
    api = wandb.Api()
    artifact=api.artifact(model_name)
    model_dir=artifact.download()
    model_path = model_dir
  else:
    model_path = checkpoint
  print('Model Path')
  print(model_path)
  # Set hyperparams in sweep configurations
  sweep_config = {
      'method': 'grid',  # can be grid, random, or bayes
      'name' : sweep_name,
      'metric': {
        'name': 'eval/f1_hierarchical',
        'goal': 'maximize'
      },
      'parameters': {
          'learning_rate' : {
              'values': [5e-6]#[5e-6, 5e-5, 5e-4]
          },
          'threshold': {
              'values': [0.4]#[0.3, 0.4, 0.5]
          },
          'run_name': {
              'value' : run_name
          }
      }
  }

  if node_name=="persuasion_or_not":
    print("Binary classification",node_name)
    sweep_config["parameters"]["threshold"]["values"]=[0.5]
    lbls=unique[0]
    id2label={0:lbls[0],1:lbls[1]}
    label2id={lbls[0]:0,lbls[1]:1}
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=len(unique[0]),id2label=id2label,label2id=label2id,ignore_mismatched_sizes=True)
  else:
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=len(unique[0]), problem_type="multi_label_classification")#, ignore_mismatched_sizes=True)
  
  print('Model')
  print(model)
  sweep_config["parameters"]["threshold"]["values"]=[threshold_nodes[node_name]]
  model.cuda()
  # Start sweeps with specific configuration
  sweep_id = wandb.sweep(sweep_config, project="subtask1_transformer_encoder_classification")
  wandb.agent(sweep_id, train)
  

  # Get best model of sweep
  api = wandb.Api()
  sweep = api.sweep(f"subtask1_transformer_encoder_classification/{sweep_id}")
  best_run = sweep.best_run()

  artifacts = best_run.logged_artifacts()

  model_artifact = None
  for artifact in artifacts:
      if 'model' in artifact.type: 
          model_artifact = artifact
          break

  if model_artifact != None:
    model_artifact_name = model_artifact.name
    print(f"Best Model for {node_name}: {model_artifact_name}")
  else:
    warnings.warn(f"No models was found")

  # save best model of this node
  model_nodes[node_name] = model_artifact_name

  # save best threshold of this node
  threshold_nodes[node_name] = extract_threshold(model_artifact_name)

Model Path
vinai/bertweet-large
Binary classification persuasion_or_not


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model
RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=T

wandb: Agent Starting Run: cmwg7bjj with config:
wandb: 	learning_rate: 5e-06
wandb: 	run_name: vinai_bertweet-large-persuasion_or_not-memes
wandb: 	threshold: 0.5


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'run_name' was locked by 'sweep' (ignored update).
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


Error in training node persuasion_or_not: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacty of 11.76 GiB of which 48.31 MiB is free. Including non-PyTorch memory, this process has 11.71 GiB memory in use. Of the allocated memory 11.36 GiB is allocated by PyTorch, and 46.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
wandb: Sorting runs by -summary_metrics.eval/f1_hierarchical
<ipython-input-33-fcf462e58fec>:89: UserWarning: No models was found
  warnings.warn(f"No models was found")


NameError: name 'model_artifact_name' is not defined

In [ ]:
!nvidia-smi

#### Save Model Config

In [ ]:
def write_json(path,data,test=False):
  if not isinstance(data, dict):
    data = data.to_dict("records")

  with open(path, "w") as output_file:
      json.dump(data, output_file, indent=2,ensure_ascii=False)

In [ ]:
summary = {
    "checkpoint" : checkpoint,
    "model_nodes" : model_nodes,
    "threshold_nodes": threshold_nodes,
    "transfer_learning": transfer_learning,
    "train_files" : dataset_memes_2024_files
}

summary

In [ ]:
summary_file_path = summary_dir_path + "summary.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)
write_json(summary_file_path, summary)

In [ ]:
summary_file_path

Delete output and tmp files

In [ ]:
def delete_dir(path):
    if os.path.exists(path):
        try:
            shutil.rmtree(path)
            print(f"The directory {path} and all its contents have been deleted successfully")
        except OSError as e:
            print(f"Error: {e.strerror}")
    else:
        print(f"The directory {path} does not exist")

In [ ]:
def delete_output_dirs(parent_directory):
    for entry in os.listdir(parent_directory):
        entry_path = os.path.join(parent_directory, entry)
        
        if os.path.isdir(entry_path) and entry.startswith('output_'):
            delete_dir(entry_path)

In [ ]:
path = folder_name + "subtask1"
delete_output_dirs(path)
path = path + "/transformer/training/"
delete_dir(path + "tumnlp")
delete_dir(path + "wandb")
delete_dir(path + "artifacts")
delete_dir(path + "tmp_trainer")

## Evaluation

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
def tokenize(examples):
    encoding = tokenizer(examples["text"], truncation=True ,padding='max_length', max_length=512)
    return encoding

In [ ]:
def get_preds(binarizer,predicted_logits,threshold=0.5):
    
  predicted_logits_tensor = torch.from_numpy(predicted_logits)

  if isinstance(binarizer,LabelEncoder):
    softmax=torch.nn.Softmax(dim=1)
    full_probs=softmax(predicted_logits_tensor)

    probs,predictions=torch.max(full_probs,dim=1)
    predictions=[[p] for p in binarizer.inverse_transform(predictions)]
    probs = [[value.item()] for value in probs]
    res=[(x, y) for x, y in zip(predictions, probs)]
    return res
  else:
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(predicted_logits_tensor)
    preds=(probs>threshold).int()

    mask = (preds == 1)

    # Use the mask to select the corresponding probabilities
    selected_probs_list = []

    # Iterate over rows and select probabilities using the mask
    for i in range(preds.size(0)):
        selected_probs_row = probs[i][mask[i]].tolist()
        selected_probs_list.append(selected_probs_row)

    names=binarizer.inverse_transform(preds)
    res=[(list(x), y) for x, y in zip(names, selected_probs_list)]
    return res

In [ ]:
project_name="subtask1_transformer_encoder_classification"

def return_trainer(model_name,unique_labels):
  api = wandb.Api()
  artifact=api.artifact(model_name)
  model_dir=artifact.download()
  if "persuasion_or_not" in model_name:
    lbls=unique_labels[0]
    id2label={0:lbls[0],1:lbls[1]}
    label2id={lbls[0]:0,lbls[1]:1}
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(unique_labels[0]),id2label=id2label, label2id=label2id)
  else:
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(unique_labels[0]), problem_type="multi_label_classification", ignore_mismatched_sizes=True)
    
  trainer = Trainer(model=model)
  if torch.cuda.is_available():
    trainer.model = model.cuda()
  return trainer

### Evaluate validation set

In [ ]:
prediction_set=val_memes #dataset_val["val"]
prediction_set=prediction_set.remove_columns(["labels"])

In [ ]:
def get_new_labels(model_address,node,unique,threshold,binarizer):
  trainer=return_trainer(model_address,unique)
  preds=get_preds(binarizer,trainer.predict(curr_set).predictions,threshold)

  for i in range(len(preds)):

    if len(preds[i][0])>0:

      t=prev_labels[i][0].index(node)
      _=prev_labels[i][1].pop(t)
      prev_labels[i][0].remove(node)
      parent_labels=prev_labels[i][0]
      parent_probs=prev_labels[i][1]

      new_labels.append((parent_labels+preds[i][0],parent_probs+preds[i][1]))
    else:
      new_labels.append(prev_labels[i])

In [ ]:
trainer=return_trainer(model_nodes["persuasion_or_not"],persuasion_or_not_unique)
threshold=threshold_nodes['persuasion_or_not']
prediction_set_tokenized=prediction_set.map(tokenize ,batched=True)

preds_=get_preds(lb_persuasion_or_not,trainer.predict(prediction_set_tokenized).predictions,threshold)
preds=[el if el[0][0]=="Persuasion" else [] for el in preds_]
final_ds=dict(zip(prediction_set_tokenized["id"],preds))

model_seq=["Persuasion","Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["persuasion","ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [persuasion_unique, ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_persuasion, mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds[x] for x in ids]
  new_labels=[]
  curr_set=prediction_set_tokenized.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  get_new_labels(model_nodes[node_names[i]],node,unique_node_labels[i],threshold_nodes[node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds[ids[k]]=new_labels[k]


### Evaluate using the scorer script

In [ ]:
def hierarchical_scores(gold_label_path, pred_label_path):
    scorer = folder_name + "subtask1/subtask_1_2a.py"
    command = f'python3 {scorer} --gold_file_path {gold_label_path} --pred_file_path {pred_label_path}'

    result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True)
    output = result.stdout.strip()

    parts = output.split('\t')
    f1_h = parts[0].split('=')[1]
    prec_h = parts[1].split('=')[1]
    rec_h = parts[2].split('=')[1]

    f1_h = float(f1_h)
    prec_h = float(prec_h)
    rec_h = float(rec_h)

    hierarchical_metrics = {"f1_hierarchical": f1_h, "precision_hierarchical": prec_h, "recall_hierarchical": rec_h}
    return hierarchical_metrics

In [ ]:
top_k=0 
hierarchical_metrics = {"f1_hierarchical": 0, "precision_hierarchical": 0, "recall_hierarchical": 0}
final_df = dict()
for curr_top_k in range(11):
    curr_final_df=dict()
    for k,v in final_ds.items():
      if len(v)>0:
        pred_prob=list(zip(v[0],v[1]))
        pred_prob.sort(key=lambda x: -x[1])

        tmp=[]
        i=0
        for el in pred_prob:
          if i==curr_top_k:
            break
          if el[0] not in tmp:
            tmp.append(el[0])
            i+=1
        curr_final_df[str(k)]=tmp
      else:
        curr_final_df[str(k)]=v
        
    curr_pred_df=pd.DataFrame.from_dict({"id":curr_final_df.keys(),"labels":curr_final_df.values()})
    curr_val_pred_file="tmp/" + "val_pred.json"
    if not os.path.exists("tmp/"):
        os.makedirs("tmp/")
    write_json(curr_val_pred_file,curr_pred_df)

    curr_hierarchical_metrics = hierarchical_scores(val_path,curr_val_pred_file)
    
    shutil.rmtree("tmp/")
    
    print(f"{curr_top_k}:")
    print(curr_hierarchical_metrics)
    if curr_hierarchical_metrics['f1_hierarchical'] >= hierarchical_metrics['f1_hierarchical']:
        hierarchical_metrics = curr_hierarchical_metrics
        top_k = curr_top_k
        final_df = curr_final_df
        
print(f"\nBest result with top_k = {top_k}")

In [ ]:
pred_df=pd.DataFrame.from_dict({"id":final_df.keys(),"labels":final_df.values()})

val_pred_file=summary_dir_path + "val_pred.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)

write_json(val_pred_file,pred_df)

hierarchical_scores(val_path,val_pred_file)

In [ ]:
summary["top_k"] = top_k
summary["Validation score"] = hierarchical_metrics

write_json(summary_file_path, summary)

### Create dev output file

In [ ]:
prediction_set=dataset_test["test"]

In [ ]:
trainer=return_trainer(model_nodes["persuasion_or_not"],persuasion_or_not_unique)

threshold=threshold_nodes['persuasion_or_not']
prediction_set_tokenized=prediction_set.map(tokenize ,batched=True)

preds_=get_preds(lb_persuasion_or_not,trainer.predict(prediction_set_tokenized).predictions,threshold)
preds=[el if el[0][0]=="Persuasion" else [] for el in preds_]
final_ds=dict(zip(prediction_set_tokenized["id"],preds))

model_seq=["Persuasion","Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["persuasion","ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [persuasion_unique, ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_persuasion, mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds[x] for x in ids]
  new_labels=[]
  curr_set=prediction_set_tokenized.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  get_new_labels(model_nodes[node_names[i]],node,unique_node_labels[i],threshold_nodes[node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds[ids[k]]=new_labels[k]


In [ ]:
final_df=dict()
for k,v in final_ds.items():
  if len(v)>0:
    pred_prob=list(zip(v[0],v[1]))
    pred_prob.sort(key=lambda x: -x[1])

    tmp=[]
    i=0
    for el in pred_prob:
      if i==top_k:
        break
      if el[0] not in tmp:
        tmp.append(el[0])
        i+=1
    final_df[str(k)]=tmp
  else:
    final_df[str(k)]=v


In [ ]:
print(model_artifact_name )
extract_threshold(model_artifact_name)

In [ ]:
pred_df=pd.DataFrame.from_dict({"id":final_df.keys(),"labels":final_df.values()})

dev_pred_file=summary_dir_path + "dev_pred.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)

write_json(dev_pred_file,pred_df)

Delete output and tmp files

In [ ]:
path = folder_name + "subtask1"
#delete_output_dirs(path)
path = path + "/transformer/training/"
delete_dir(path + "tumnlp")
delete_dir(path + "wandb")
delete_dir(path + "artifacts")
delete_dir(path + "tmp_trainer")

In [ ]:
summary

In [ ]:
summary_file_path